In [72]:
#PART 1

In [1]:
# Install boto3 using pip 
## Add '!' only if you install directly from a Jupyter Notebook
%pip install Boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 19.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninstalled botocore-1.27.59
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.0 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.27.96 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import json
import os
import io
import pandas as pd
import boto3
import glob
import plotly_express as px

In [26]:
#Create an instance of boto3.Session that connects with your aws account.
import boto3
session = boto3.Session(aws_access_key_id="", 
                        aws_secret_access_key="")

In [9]:
s3 = session.client("s3") #Create a variable called s3 that connects your session to the s3 client to upload files to your bucket.

In [11]:
bucket = s3.create_bucket(Bucket="projectbookingbasmas3", CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [5]:
file1 = '/Users/fezzibasma/Desktop/Data/Data_Full-Stack/Data Collection & Management/ProjetBooking/III) tables_extract_csv/booking_bestpercitycenter.csv'
file2 = '/Users/fezzibasma/Desktop/Data/Data_Full-Stack/Data Collection & Management/ProjetBooking/III) tables_extract_csv/gps_cities.csv'
file3= '/Users/fezzibasma/Desktop/Data/Data_Full-Stack/Data Collection & Management/ProjetBooking/III) tables_extract_csv/meteo_ofcities_7days.csv'
csv_files=[file1,file2,file3]

In [6]:
for filename in csv_files:
    key = "%s/%s" % ('III) tables_extract_csv', os.path.basename(filename))
    print("Putting %s as %s" % (filename,key))
    s3.upload_file(filename, "projectbookingbasmas3" , key)

Putting /Users/fezzibasma/Desktop/Data/Data_Full-Stack/Data Collection & Management/ProjetBooking/III) tables_extract_csv/booking_bestpercitycenter.csv as III) tables_extract_csv/booking_bestpercitycenter.csv
Putting /Users/fezzibasma/Desktop/Data/Data_Full-Stack/Data Collection & Management/ProjetBooking/III) tables_extract_csv/gps_cities.csv as III) tables_extract_csv/gps_cities.csv
Putting /Users/fezzibasma/Desktop/Data/Data_Full-Stack/Data Collection & Management/ProjetBooking/III) tables_extract_csv/meteo_ofcities_7days.csv as III) tables_extract_csv/meteo_ofcities_7days.csv


In [8]:
#PART 2:

In [12]:
df_booking=pd.read_csv('https://s3.console.aws.amazon.com/s3/object/projectbookingbasmas3?region=eu-west-3&prefix=III%29+tables_extract_csv/booking_bestpercitycenter.csv')
df_weather_ofcities=pd.read_csv('https://s3.console.aws.amazon.com/s3/object/projectbookingbasmas3?region=eu-west-3&prefix=III%29+tables_extract_csv/meteo_ofcities_7days.csv')

## Pull S3 to get object

In [13]:
import csv

#Import des hôtels booking : 
obj_booking = s3.get_object(Bucket='projectbookingbasmas3', Key='III) tables_extract_csv/booking_bestpercitycenter.csv') #2
data_booking = obj_booking['Body'].read().decode('utf-8').splitlines() #3
records_booking = csv.reader(data_booking) #4
headers_booking = next(records_booking) #5

#Import de la météo et gps des villes françaises : 
obj_weather = s3.get_object(Bucket='projectbookingbasmas3', Key='III) tables_extract_csv/meteo_ofcities_7days.csv') #2
data_weather = obj_weather['Body'].read().decode('utf-8').splitlines() #3
records_weather = csv.reader(data_weather) #4
headers_weather = next(records_weather) #5


# #Lectures des deux DataFrames : 
df_booking=pd.DataFrame(records_booking,columns=headers_booking)
df_weather_ofcities=pd.DataFrame(records_weather,columns=headers_weather)

In [14]:
response = s3.get_object(Bucket='projectbookingbasmas3', Key='III) tables_extract_csv/booking_bestpercitycenter.csv')

status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 get_object response. Status - {status}")
    books_df = pd.read_csv(response.get("Body"))
    print(books_df)
else:
    print(f"Unsuccessful S3 get_object response. Status - {status}")

Successful S3 get_object response. Status - 200
                                 hotelname  \
0                Mercure Mont Saint Michel   
1                   La Parenthèse Envoutée   
2                                 The Nest   
3                              Moxy Amiens   
4                ibis Styles Amiens Centre   
..                                     ...   
502             Novotel La Rochelle Centre   
503            ibis La Rochelle Vieux Port   
504                     Hôtel François 1er   
505  Hôtel Kyriad La Rochelle Centre Ville   
506                   Un Hotel sur le Port   

                                     location  \
0                        Le Mont-Saint-Michel   
1                                      Amiens   
2                                      Amiens   
3                                      Amiens   
4                                      Amiens   
..                                        ...   
502                               La Rochelle   
503  Ce

In [83]:
# PART 3 : Data Formatting columns for join 

In [15]:
#Reformattage des intitulés de colonnes de Booking pour EDA

df_booking.columns=['Booking Hôtel','Localisation détaillée','Description Hôtel ','Notation','Location_splitted','Ville']
df_booking.drop(['Location_splitted'],axis=1,inplace=True)
df_booking

,Booking Hôtel,Localisation détaillée,Description Hôtel,Notation,Ville
0,Mercure Mont Saint Michel,Le Mont-Saint-Michel,Installé dans des espaces verts à seulement 2 ...,8.2,Mont Saint Michel
1,La Parenthèse Envoutée,Amiens,"Situé à Amiens, en Picardie, à proximité de la...",8.9,Amiens
2,The Nest,Amiens,"Situé à 1,7 km du Zénith d'Amiens, l'établisse...",8.5,Amiens
3,Moxy Amiens,Amiens,Situé au cœur de la ville et à deux pas de la ...,8.5,Amiens
4,ibis Styles Amiens Centre,Amiens,"Situé à Amiens, à 3,4 km du Zénith, l'ibis Sty...",8.7,Amiens
...,...,...,...,...,...
502,Novotel La Rochelle Centre,La Rochelle,Le Novotel La Rochelle Centre est situé à 10 m...,8.2,La Rochelle
503,ibis La Rochelle Vieux Port,"Centre-ville de La Rochelle, La Rochelle","Situé dans le quartier du Vieux-Port, dans le ...",8.1,La Rochelle
504,Hôtel François 1er,"Centre-ville de La Rochelle, La Rochelle","Situé dans le cœur historique de La Rochelle, ...",8.1,La Rochelle
505,Hôtel Kyriad La Rochelle Centre Ville,La Rochelle,"Affichant une décoration moderne, l'Hôtel Kyri...",8.1,La Rochelle


In [16]:
#Verification de l'unicité de la colonne ligne entre les deux datasets
len(df_booking.Ville.unique())

35

In [17]:
# #Reformattage des intitulés de colonnes de Weather pour EDA

df_weather_ofcities.columns=['index','day','day_temperature','day_rain','Ville','latitude','longitude']
df_weather_ofcities.drop(['index'],axis=1,inplace=True)


In [18]:
#Verification de l'unicité de la colonne ligne entre les deux datasets
len(df_weather_ofcities.Ville.unique())

35

In [19]:
#Merge les deux DataFrames dans une grande DataFrame pour EDA 

df_final=df_booking.merge(df_weather_ofcities, on='Ville', how='left')
df_final.head()

,Booking Hôtel,Localisation détaillée,Description Hôtel,Notation,Ville,day,day_temperature,day_rain,latitude,longitude
0,Mercure Mont Saint Michel,Le Mont-Saint-Michel,Installé dans des espaces verts à seulement 2 ...,8.2,Mont Saint Michel,2022-09-29,15.1,0.0,48.62,-1.5
1,Mercure Mont Saint Michel,Le Mont-Saint-Michel,Installé dans des espaces verts à seulement 2 ...,8.2,Mont Saint Michel,2022-09-29,15.1,0.0,48.62,-1.5
2,Mercure Mont Saint Michel,Le Mont-Saint-Michel,Installé dans des espaces verts à seulement 2 ...,8.2,Mont Saint Michel,2022-09-29,15.1,0.0,48.62,-1.5
3,Mercure Mont Saint Michel,Le Mont-Saint-Michel,Installé dans des espaces verts à seulement 2 ...,8.2,Mont Saint Michel,2022-09-30,16.0,0.7,48.62,-1.5
4,Mercure Mont Saint Michel,Le Mont-Saint-Michel,Installé dans des espaces verts à seulement 2 ...,8.2,Mont Saint Michel,2022-09-30,16.0,0.7,48.62,-1.5


In [20]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [21]:
db_final=df_final.to_sql('df_final', con=engine)

In [22]:
#PARTIE POUR LEFT JOIN EN SQL DIRECTEMENT AU LIEU DE PANDAS ( OPTIONNEL)


# df_weather_ofcities.to_sql('df_weather_ofcities', con=engine)
# #notre DB final est : 
# from sqlalchemy.sql import text
# query="SELECT a.*,b.day,b.day_temperature,b.day_rain,b.latitude,b.longitude FROM df_booking a LEFT JOIN df_weather_ofcities b on (a.Ville=b.Ville) "
# #READ SQL BASE AND CONVERT TO DATAFRAME FOR EDA
# df_final=pd.read_sql(query,engine)


In [94]:
#PART 4 EDA 

In [23]:
df_final.loc[df_final['Notation'].isnull()] #verifier si les ratings ont bien été attribués en selectionnant les lignes Nan

,Booking Hôtel,Localisation détaillée,Description Hôtel,Notation,Ville,day,day_temperature,day_rain,latitude,longitude


In [24]:
df_weather_ofcities.loc[df_weather_ofcities['day_temperature'].isnull()]

,day,day_temperature,day_rain,Ville,latitude,longitude


In [25]:
#On reformate les colonnes numériques en float au lieu str pour afficher les graphiques
df_weather_ofcities = df_weather_ofcities.astype({'day_temperature':'float','day_rain':'float','latitude':'float','longitude':'float'})
df_final = df_final.astype({'Notation':'float','day_temperature':'float','day_rain':'float','latitude':'float','longitude':'float'})


In [98]:
import plotly.io as pio
import plotly.express as px

In [99]:
fig = px.scatter_mapbox(df_final, lat="latitude", lon="longitude", color="day_temperature", size="day_temperature",zoom=5, mapbox_style="carto-positron",color_continuous_scale=px.colors.cyclical.IceFire, size_max=30,opacity=0.2)
fig.show()

In [27]:
df_top5destinations=df_weather_ofcities.sort_values(by=['day_temperature'],ascending=False).drop_duplicates().reset_index().drop(['index'], axis=1)
df_top5destinations=df_top5destinations.head(30)
df_top5destinations['Ville'].unique()

array(['Aigues Mortes', 'Saintes Maries de la mer', 'Nimes', 'Montauban',
       'Avignon', 'Uzes', 'Carcassonne', 'Colmar', 'Toulouse',
       'Aix en Provence', 'Bormes les Mimosas', 'Strasbourg', 'Eguisheim',
       'Cassis', 'Grenoble', 'Lyon'], dtype=object)

## TOP-5 DESTINATIONS 

In [101]:
df_top5destinations = px.scatter_mapbox(df_top5destinations, lat="latitude", lon="longitude", color="day_temperature", size="day_temperature",zoom=5, mapbox_style="carto-positron",color_continuous_scale=px.colors.cyclical.IceFire,opacity=0.2)
df_top5destinations.show()

In [28]:
Liste_topvilles=df_top5destinations['Ville'].unique()

In [29]:
df_tophotelsinarea=df_final[df_final['Ville'].isin(Liste_topvilles)].sort_values(by=['Notation'],ascending=False).drop_duplicates().reset_index().drop(['index'], axis=1)
Liste_20besthotels=df_tophotelsinarea['Booking Hôtel'].unique()[:20]
len(Liste_20besthotels)

20

In [30]:
df_20tophotelsinarea=df_tophotelsinarea[df_tophotelsinarea['Booking Hôtel'].isin(Liste_20besthotels)]
df_20tophotelsinarea

,Booking Hôtel,Localisation détaillée,Description Hôtel,Notation,Ville,day,day_temperature,day_rain,latitude,longitude
0,Boutique Hotel SOCLO,"Centre de Toulouse, Toulouse","Situé à Toulouse, le Boutique Hotel SOCLO poss...",9.6,Toulouse,2022-10-01,22.3,0.0,43.600000,1.44
1,Boutique Hotel SOCLO,"Centre de Toulouse, Toulouse","Situé à Toulouse, le Boutique Hotel SOCLO poss...",9.6,Toulouse,2022-10-02,23.4,0.0,43.600000,1.44
2,Boutique Hotel SOCLO,"Centre de Toulouse, Toulouse","Situé à Toulouse, le Boutique Hotel SOCLO poss...",9.6,Toulouse,2022-10-05,25.5,0.0,43.600000,1.44
3,Boutique Hotel SOCLO,"Centre de Toulouse, Toulouse","Situé à Toulouse, le Boutique Hotel SOCLO poss...",9.6,Toulouse,2022-10-04,24.8,0.0,43.600000,1.44
4,Boutique Hotel SOCLO,"Centre de Toulouse, Toulouse","Situé à Toulouse, le Boutique Hotel SOCLO poss...",9.6,Toulouse,2022-10-03,24.2,0.0,43.600000,1.44
...,...,...,...,...,...,...,...,...,...,...
143,"InterContinental Lyon - Hotel Dieu, an IHG Hotel","2e arr., Lyon","Situé à Lyon, l'InterContinental Lyon - Hotel ...",9.1,Lyon,2022-10-01,15.6,4.2,45.760002,4.84
144,"InterContinental Lyon - Hotel Dieu, an IHG Hotel","2e arr., Lyon","Situé à Lyon, l'InterContinental Lyon - Hotel ...",9.1,Lyon,2022-09-29,15.3,5.5,45.760002,4.84
145,"InterContinental Lyon - Hotel Dieu, an IHG Hotel","2e arr., Lyon","Situé à Lyon, l'InterContinental Lyon - Hotel ...",9.1,Lyon,2022-09-30,15.0,0.0,45.760002,4.84
146,Villa Gallici Hôtel & Spa,Aix-en-Provence,"Situé cœur de la Provence, le Villa Gallici Hô...",9.1,Aix en Provence,2022-10-04,24.9,0.0,43.520000,5.44


## TOP-20 HOTELS IN THE AREA 

In [45]:
fig = px.scatter_mapbox(df_20tophotelsinarea, lat="latitude", lon="longitude",text="Notation",color="Booking Hôtel",size="day_temperature",zoom=5,mapbox_style="carto-positron",opacity=1,size_max=100)
fig.show()